# Cross-validation and hyperparameter selection
For this project, we are going to explore the cross-validation testing methodology. We'll use it to get error estimates on two algorithms we've presented in class:
  - Linear Regression
  - Decision Tree classification

You'll also get to implement a much more precise method than gradient descent to compute the optimal coefficients for linear regression, using the **least-squares** method.
  

In [129]:
# These are the libraries you will use for this project.
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1
**Calculate Generalized Error on Linear Regression with $k$-fold Cross Validation**


## Q1.1
**Load in the scikit-learn `diabetes` data set.**

Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html). 
- Name your feature DataFrame (the independent variables) `df_X`.
- Name your target series (the dependent variable) `s_y`.


<!-- BEGIN QUESTION -->



In [152]:
df_X, s_y = load_diabetes(return_X_y=True,as_frame=True)
# The sizes should be (442, 10) and (442,).
s_y = s_y.to_numpy()
#print(type(df_X), type(s_y))
print(df_X.shape, s_y.shape)

(442, 10) (442,)


<!-- END QUESTION -->

## Q1.2 

**Define a function that creates a linear least-squares regression model**

- Your function should take in two parameters:
    - `A` is the feature DataFrame (i.e. the `df_X` you got in Q1.1.)
    - `b` is the target Series (i.e. the `s_y` you got in Q1.1.)
- Your function should return a least-squares solution $x$ to the linear matrix equation $Ax = b$.
    - $A$ contains the features for each sample. (Based on `df_X` in Q1.1)
    - $x$ (the solution) is a vector of coefficients for each feature (column) in $A$. (for you to find)
    - $b$ (the target) contains the target values of each sample. (Based on `s_y` in Q1.1)
    - You should also fit an intercept to your function. You can do this by augmenting `df_X` ($A$) with an additional column of `1`s. Name this column `intercept` (all lower case).
- Once you get $x$ from your function, you will use it in later parts as the coefficients in $x$ to form a linear regression model to estimate the target value for any set of feature values.

Notes:
-  You can not use any libraries outside of pandas and numpy. 
- *Hint*: The following function may be useful: [numpy.linalg.lstsq](https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html)


<!-- BEGIN QUESTION -->



In [153]:
def add_intercept(df_X: pd.DataFrame) -> pd.DataFrame:
    """Copy a DataFrame. Then, add an intercept column of just '1's to
    the copy. Return the improved DataFrame."""
    df_X_copy = df_X
    df_X_copy["intercept"] = 1
    return df_X_copy

def linear_regression(A: pd.DataFrame, b: pd.Series) -> np.ndarray:
    """Generate a least-squares linear regression model given a DataFrame of
    features and a Series of target values. 
    
    Do not add an intercept to A inside this function. It should be done
    before passing it in.

    You should return x, the coefficients of the linear regression model.
    """
    x = np.linalg.lstsq(A,b,rcond=None)
    return x[0]


In [155]:
# Add an intercept to df_X
df_X_intercept = add_intercept(df_X)
#print(df_X_intercept)

# Compute your least squares solution
x = linear_regression(df_X_intercept, s_y)
#x = x[0]

# The shape of x should be (11,).
print(x, x.shape)

[ -10.01219782 -239.81908937  519.83978679  324.39042769 -792.18416163
  476.74583782  101.04457032  177.06417623  751.27932109   67.62538639
  152.13348416] (11,)


<!-- END QUESTION -->

## Q1.3 

**Define a function that partitions the dataframe and series data into dictionaries**

- Your function should take in three parameters:
    - `df_X`, as before
    - `s_y`, as before
    - `k`, the number of partitions
- Your function should return two dictionaries:
    - `dict_k_X`: Partitions of the feature DataFrame
    - `dict_s_y`: Partitions of the target Series
    - For each dictionary:
        - *Keys* are the value `k`, i.e. the partition number (1 to `k` inclusive)
        - *Values* are the features DataFrame/target Series of the samples *inside that partition* (make sure the features match the targets match.)

`dict_k_X` will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Likewise, `dict_s_y` will have the series of the target data that contains approximately $\frac{1}{k}$ of the data. Please note the **features and targets must match the same rows in the DataFrame**.

Finally, call that function with $k=5$ and create the dictionaries `dict_k_X` and `dict_k_y`. 
- Make sure you use `df_X_intercept` from Q1.2 so you can apply that to your least-squares model.
- Print out the number of rows in each fold.  
- Check that the number of data points in each partition totals the number of data points in the entire dataset. 


<!-- BEGIN QUESTION -->



In [156]:
def partition_data(df_X: pd.DataFrame, s_y: pd.Series, k: int) -> (dict, dict):
    """Partition the dataset into k folds (partitions). 
    
    You should return two dictionaries, which contain the k partitions 
    over the feature DataFrame and target value Series.
    """
    df_l = int(len(df_X)/k) #88
    df_m = len(df_X)%k # 2
    
    sy_l = int(len(s_y)/k) #88
    sy_m = len(s_y)%k #2
    
    dict_k_X = {}
    dict_k_y = {}
    
    for i in range(1,k+1):
        if(i == k):
            dict_k_X[i] = df_X.iloc[(i-1)*df_l: (i *df_l) + df_m]
            dict_k_y[i] = s_y[(i-1)*sy_l: (i * sy_l) + sy_m]
            #print((i-1)*df_l)
            #print((i *df_l) + df_m)
            
        else:
            dict_k_X[i] = df_X.iloc[(i-1)*df_l: i *df_l]
            dict_k_y[i] = s_y[(i-1)*sy_l: i * sy_l]
            #print((i-1) *df_l)
            #print(i *df_l)
    

    # Construct the dictionaries.
    return (dict_k_X, dict_k_y)

In [72]:
#df_X_intercept.iloc[352:442]

In [158]:
# Generate the k folds.
dict_k_X, dict_k_y = partition_data(df_X_intercept, s_y, 5)
#print(dict_k_X)

# Check the number of rows in each fold.
for k in dict_k_X.keys():
    print(f'fold k: {k}, features: {len(dict_k_X[k])}, targets: {len(dict_k_y[k])}')

fold k: 1, features: 88, targets: 88
fold k: 2, features: 88, targets: 88
fold k: 3, features: 88, targets: 88
fold k: 4, features: 88, targets: 88
fold k: 5, features: 90, targets: 90


<!-- END QUESTION -->

## Q1.4 

**Define a function that calculates a regression metric / error function**

- Your function should take two Numpy arrays (You may need to cast your input by calling `.to_numpy()` first):
    - `s_y`: The target values 
    - `s_y_hat`: A series of predictions on the samples for `s_y`.
    - The lengths should be equal.
- You should calcuate and return the **mean absolute error (MAE)** between `s_y` and `s_y_hat`:
    - $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```



<!-- BEGIN QUESTION -->



In [159]:
def get_mae(s_y: np.ndarray, s_y_hat: np.ndarray) -> float:
    """Compute the mean absolute error between the true values s_y
    and the predicted values s_y_hat."""
    s_y, s_y_hat = np.array(s_y), np.array(s_y_hat)
    mae = np.mean(np.abs(s_y - s_y_hat))
    return mae

In [160]:
# Test your function. You should see approximately 0.33.
y_true_sample, y_pred_sample = np.array([1,2,3]), np.array([2,2,3])
mae_sample = get_mae(y_true_sample, y_pred_sample)
print('MAE:', mae_sample)

MAE: 0.3333333333333333


<!-- END QUESTION -->

## Q1.5 

**Calculate the $MAE$ for each fold**

For each of your $k = 5$ folds:
1. Split your dataset into a training and testing set as follows:
    - The current fold is withheld as the testing set.
    - The remaining four folds are used for the training set.
    - (Use the partition number key as the test set, and all other partitions as the train set.)
2. Train a linear regression model on this fold's training set, using Q 1.2
3. Test your model on the testing fold, by multiplying $Ax$ on the coefficients you found. ($Ax = \hat{b}$, which we will compare to the true targets $b$)
4. Compute the MAE of your test predictions vs. the test targets.

Store the five MAEs inside the `mae` array, and print the min, max, and mean $MAE$ of your 5 folds. 

Make sure your `dict_k_X` folds each contain an `intercept` column (this should have already been done for you in Q1.3).


<!-- BEGIN QUESTION -->



In [185]:
mae = {} # Use this array to track the MAE of your 5 folds.
#dict_k_X, dict_k_y

# Place your folds inside these dictionaries:
dict_k_X_train = {}
dict_k_y_train = {}
dict_k_X_test = {}
dict_k_y_test = {}

for k in dict_k_X.keys():
    # Construct the testing and training sets:
    df_X_train = pd.DataFrame()
    s_y_train = pd.DataFrame()
    
    df_X_test = dict_k_X[k]
    s_y_test = dict_k_y[k]
    
    for j in range(1,6):
        if(j != k):
            df_X_train = df_X_train.append(dict_k_X[j])
            ss = pd.DataFrame(data=dict_k_y[j])
            s_y_train = s_y_train.append(ss)
    
    # Compute the linear regression coefficients x_k on the training set, then
    # the predictions s_y_hat on the test set.
    
    x_k = linear_regression(df_X_train, s_y_train.squeeze())
    s_y_hat = get_mae(df_X_test, x_k)
    
    # Compute the MAE for the fold.
    mae_k = get_mae(s_y_test, s_y_hat)

    # Add your folds/MAEs to the dictionaries. (Don't edit these five lines.)
    mae[k] = mae_k
    dict_k_X_train[k] = df_X_train
    dict_k_y_train[k] = s_y_train
    dict_k_X_test[k] = df_X_test
    dict_k_y_test[k] = s_y_test

    # You may find the following print statements useful for debugging.
    print('Fold k:', k)
    print('X_train, y_train:', df_X_train.shape, s_y_train.shape)
    print('X_test, y_test  :', df_X_test.shape, s_y_test.shape)
    print('x_k             :', x_k.shape)
    print('y_test_hat      :', s_y_hat.shape)
    print('mae             :', mae_k)
    print()

# At the end, print out the MAEs and statistics.
print('MAEs: ', list(mae.values()))
print('min: ', min(mae.values()))
print('max: ', max(mae.values()))
print('mean: ', sum(mae.values())/len(mae.values()))

Fold k: 1
X_train, y_train: (354, 11) (354, 1)
X_test, y_test  : (88, 11) (88,)
x_k             : (11,)
y_test_hat      : ()
mae             : 179.1567252665175

Fold k: 2
X_train, y_train: (354, 11) (354, 1)
X_test, y_test  : (88, 11) (88,)
x_k             : (11,)
y_test_hat      : ()
mae             : 151.2852885045639

Fold k: 3
X_train, y_train: (354, 11) (354, 1)
X_test, y_test  : (88, 11) (88,)
x_k             : (11,)
y_test_hat      : ()
mae             : 214.36923172734748

Fold k: 4
X_train, y_train: (354, 11) (354, 1)
X_test, y_test  : (88, 11) (88,)
x_k             : (11,)
y_test_hat      : ()
mae             : 238.06197653073949

Fold k: 5
X_train, y_train: (352, 11) (352, 1)
X_test, y_test  : (90, 11) (90,)
x_k             : (11,)
y_test_hat      : ()
mae             : 108.51438407821526

MAEs:  [179.1567252665175, 151.2852885045639, 214.36923172734748, 238.06197653073949, 108.51438407821526]
min:  108.51438407821526
max:  238.06197653073949
mean:  178.27752122147672


C:\Users\tk_42\AppData\Local\Temp\ipykernel_13088\949510406.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_X_train = df_X_train.append(dict_k_X[j])
C:\Users\tk_42\AppData\Local\Temp\ipykernel_13088\949510406.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  s_y_train = s_y_train.append(ss)
C:\Users\tk_42\AppData\Local\Temp\ipykernel_13088\949510406.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_X_train = df_X_train.append(dict_k_X[j])
C:\Users\tk_42\AppData\Local\Temp\ipykernel_13088\949510406.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  s_y_train = s_y_train.append(ss)
C:\Users\tk_42\AppData\Local\Temp\ipyk

<!-- END QUESTION -->

# Part 2

**Find the best hyperparameter to use in a Decision Tree**


## Q2.1 

**Load in the scikit-learn `iris` data set**
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). 
- Name your features DataFrame (the independent variables) `df_X`.
- Name your target series (the dependent variable) series `s_y`.


<!-- BEGIN QUESTION -->



In [98]:
df_X, s_y = load_iris(return_X_y=True,as_frame=True)
s_y = s_y.to_numpy()

# The sizes should be (150, 4) and (150,).
print(df_X.shape, s_y.shape)

(150, 4) (150,)


<!-- END QUESTION -->

## Q2.2 

**Partition `df_X` and `s_y` into $5$ partitions of roughly equal size**

Just like in Q1.3, partition `df_X` and `s_y` into $k = 5$ partitions (folds) of roughly equal size. Use the `partition_data` function you wrote in Q1.3 to achieve this.

As a reminder, you should generate these dictionaries for the iris dataset:
- `dict_k_X`: Partitions of the feature DataFrame
- `dict_s_y`: Partitions of the target Series
- For each dictionary:
    - *Keys* are the value `k`, i.e. the partition number (1 to `k` inclusive)
    - *Values* are the features DataFrame/target Series of the samples *inside that partition* (make sure the features match the targets match.)

Do not add an intercept, as we're working with a decision tree model this time.


<!-- BEGIN QUESTION -->



In [105]:
dict_k_X, dict_k_y = partition_data(df_X, s_y, 5)
#print(df_X)
#print(s_y)
# Check the number of rows in each fold.
for k in dict_k_X.keys():
    print(f'fold k: {k}, features: {len(dict_k_X[k])}, targets: {len(dict_k_y[k])}')

fold k: 1, features: 30, targets: 30
fold k: 2, features: 30, targets: 30
fold k: 3, features: 30, targets: 30
fold k: 4, features: 30, targets: 30
fold k: 5, features: 30, targets: 30


<!-- END QUESTION -->

## Q2.3 

**Define a function that calculates accuracy**
The iris dataset has a categorical target, so we can compute accuracy by checking if each prediction and target are equal. The accuracy is the number of equal elements divided by the total number of elements.

- Your function should take two Numpy arrays (You may need to cast your input by calling `.to_numpy()` first):
    - `s_y`: The target values 
    - `s_y_hat`: A series of predictions on the samples for `s_y`.
    - The lengths should be equal.
- You should calcuate and return the **accuracy** of `s_y_hat` against `s_y`.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 


<!-- BEGIN QUESTION -->



In [175]:
def get_accuracy(s_y: np.ndarray, s_y_hat: np.ndarray) -> float:
    """Compute the accuracy of the predicted values s_y_hat, against 
    the true values s_y."""
    count = 0
    for i in range(len(s_y)):
        if(s_y[i] == s_y_hat[i]):
            count += 1
    
        
    acc = count/len(s_y)
    return acc

In [107]:
# Test your function. You should see approximately 0.33.
get_accuracy(s_y, np.ones(len(s_y)))

0.3333333333333333

<!-- END QUESTION -->

## Q2.4 

**Using nested cross-validation, find the best hyperparameter for a decision tree**

- Create an outer loop for 5-fold cross validation. For each outer fold:
	- Our goal is to pick the best `min_impurity_decrease` for this fold, and train a tree on it.
	- We will use a few choices for `min_impurity_decrease`: $0.1, 0.25, 0.3, 0.4$.
	- For each choice of `min_impurity_decrease`:
		- Create an inner loop with 4-fold cross validation. For each inner fold:
			- Use the scikit-learn [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree with the current choice of `min_impurity_decrease` we are considering.
			- Use the Gini Index as your impurity measure / criterion.
			- *Hint*: We use 4-fold cross validation in the inner loop, so you don't have to re-partition the training set again.
			- *Hint*: Your 4-folds should not change between hyperparameter choices.
		- Calculate the mean accuracy across the 4 inner folds for the candidate `min_impurity_decrease` values.
	- Pick the most accurate `min_impurity_decrease` from the inner loops. If there's a tie, you can pick any of them.
	- Build another DecisionTreeClassifier, and train/test it on the outer fold.

Print the following:
- For each outer fold:
	- The accuracy of the inner decision trees over the four inner folds for each `min_impurity_decrease` choice.
	- The best (most-accurate) `min_impurity_decrease` as determined in the inner loop.
	- The accuracy of the outer decision tree for the best `min_impurity_decrease`.

Your code will be a triply nested loop of the form:
```
- for k_outer in folds:
  - for choice in min_impurity_decrease:
    - for k_inner in inner folds:
```

Your output can look something like this (values will not be exact):
```
Fold 1:
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 94.06%
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 86.30%
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 63.50%
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 27.15%

Best min impurity decrease is 0.1
Accuracy is 94.4%

Fold 2:

...

```

You should also track the accuracy of each of the outer folds, to help you in Q2.5.


<!-- BEGIN QUESTION -->



In [ ]:
# Choices of min_impurity_decrease
min_impurity_decrease_choices = np.array([0.1, 0.25, 0.3, 0.4])

# Outer loop: 5-fold cross validation
for k in dict_k_X.keys():
    # Construct the testing and training sets
    df_X_train = ...
    s_y_train = ...

    df_X_test = ...
    s_y_test = ...

    
    # Middle loop: Sweep across hyperparamter values
    for min_impurity_decrease in min_impurity_decrease_choices:

        # Select the 4 folds (i.e. the 4 k-values) we will use for the inner loop.
        inner_ks = ...

        # Inner loop: 4-fold cross validation
        for k_inner in inner_ks:
            # Construct the testing and training sets
            df_X_train_inner = ...
            s_y_train_inner = ...

            df_X_test_inner = ...
            s_y_test_inner = ...

            
            # Create and train the decision tree, with the given hyperparameter value
            clf = ...
            
            # Compute the accuracy of this decision tree on the inner testing fold.
            acc_inner = ...
        

    # Compute the most accuracy min impurity decrease over the 4-folds.
    best_min_impurity_decrease = ...
    
    # Create the decision tree using the best min impurity decrease
    clf = ...
    clf = clf.fit(df_X_train, s_y_train)
            
    # Compute the accuracy of this decision tree on the outer testing fold.
    acc = ...
    


<!-- END QUESTION -->

## Q2.5 

**Show the generalized performance of the classifier** by printing the minimum, maximum, and average accuracy over the outer fold test sets.


<!-- BEGIN QUESTION -->



In [ ]:
...

<!-- END QUESTION -->

